# Installing Dependencies

In [1]:
import numpy as np
import cv2
import face_recognition
import os
#from matplotlib import pyplot as plt

# Reading Dataset Image

In [2]:
def read_img(path):
   img = cv2.imread(path)
   (h, w) = img.shape[:2]
   width = 400
   ratio = width / float(w)
   height = int(h * ratio)
   return cv2.resize(img, (width, height))

dataset_encodings = []
dataset_names = []
face_enc = []
dataset_dir = 'dataset'

for file in os.listdir(dataset_dir):
    img = read_img(dataset_dir + '/' + file)
    img_enc = face_recognition.face_encodings(img)[0]
    if len(img_enc) > 1:
        print("WARNING: More than one face found in {}. Only considering the first face.".format(file))
    if len(img_enc) == 0:
        print("WARNING: No faces found in {}. Ignoring file.".format(file))
    else:
        dataset_encodings.append(img_enc)
        dataset_names.append(file.split('.')[0])
 
 
#for i in range(0,len(dataset_encodings)):
#    face_enc.append(dataset_encodings[i][0])

In [3]:
dataset_names

['shrijon']

In [5]:
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(dataset_encodings, face_encoding)

        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
             first_match_index = matches.index(True)
             name = dataset_names[first_match_index]

        # Or instead, use the known face with the smallest distance to the new face
        #face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        #best_match_index = np.argmin(face_distances)
        #if matches[best_match_index]:
        #    name = known_face_names[best_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()